## Get data and import libraries

In [ ]:
!git clone https://github.com/DariaRev/project_autobrea

Cloning into 'project_autobrea'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 26 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), done.


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 8.2 MB 65.8 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=2c1f80e21dfd314adb0b7e2c6b279a33eeedf279a37c56055e8013c7d00924ff
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
!pip install stanza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 23.4 MB/s 
     |████████████████████████████████| 240 kB 80.3 MB/s 
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=b392a41ad25173e6d605c0bbc8e0b62022afc1db96134af2cf56d64022d81f9c
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings(action = 'ignore')
import gensim
from gensim.models import Word2Vec
from pymorphy2 import MorphAnalyzer

In [ ]:
import gensim, logging
from tqdm.auto import tqdm

In [ ]:
m = MorphAnalyzer()
from collections import defaultdict
aspect2cat = defaultdict(list)
with open('project_autobrea/data/train_split_aspects.txt', encoding = 'utf-8') as f:
    text = f.readlines()
    for line in text:
        line = line.rstrip('\r\n').split('\t')
        cat = line[1]
        ana = m.parse(line[2])[0]
        lemma = ana.normal_form
        aspect2cat[lemma].append(cat)


In [ ]:
from collections import Counter
aspect2cat_fin = {asp: Counter(l).most_common(1)[0][0] for asp, l in aspect2cat.items()}

## Preprocessing

In [ ]:
m = MorphAnalyzer()
def preprocess_aspects(file, with_cats = False):
    all_aspects = []
    with open(file, encoding = 'utf-8') as f:
        text = f.readlines()
        for line in text:
            line = line.rstrip('\r\n').split('\t')
            words = line[2].split(' ')
            asp_norm = ''
            for word in words:
                ana = m.parse(word)[0]
                if ana.tag.POS:
                    aspect = ana.normal_form + '_' + ana.tag.POS
                else:
                    aspect = ana.normal_form
                if not with_cats:
                    if aspect not in all_aspects:
                        all_aspects.append(aspect)
                if with_cats:
                    if len(words) == 1:
                        asp_norm = aspect
                    else:
                        aspect_new = aspect.split('_')[0]
                        asp_norm += aspect_new + '::'
            if len(words) > 1:
                asp_norm = asp_norm[:-2]
                asp_norm += f'_{m.parse(words[-1])[0].tag.POS}'
            if with_cats:
                all_aspects.append((line[1], asp_norm.strip()))


            
    return all_aspects

# First approach of selecting aspects in reviews and assessing sentiment by aspects and categories.

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
train_reviews = {}
test_reviews = {}
with open('project_autobrea/data/train_split_reviews.txt', encoding = 'utf-8') as f:
    text = f.read()
    for ind, line in enumerate(text.splitlines()):
        num, text = line.split('\t')
        train_reviews[num] = text
with open('project_autobrea/data/dev_reviews.txt', encoding = 'utf-8') as f:
    text = f.read()
    for ind, line in enumerate(text.splitlines()):
        num, text = line.split('\t')
        test_reviews[num] = text       

In [ ]:
aspects2reviews_test = defaultdict(list)
with open('project_autobrea/data/dev_aspects.txt', encoding = 'utf-8') as f:
    text = f.read()
    for ind, line in enumerate(text.splitlines()):
        line = line.rstrip('\r\n').split('\t')
        num = line[0]
        if len(line[2].split(' ')) == 1:
            ana = m.parse(line[2])[0]
            aspect = ana.normal_form
            aspects2reviews_test[num].append(aspect)

## 2nd approach of aspects selection

#### Define all aspects and categories to which they belong

In [ ]:
asp_li = preprocess_aspects('project_autobrea/data/train_split_aspects.txt', with_cats = True)

In [ ]:
asp_set = list(set([tuple(t) for t in asp_li ]))

#### Use word2vec on 1-grams, do nothing with n-grams

In [ ]:
import gensim.downloader as api

model = api.load("word2vec-ruscorpora-300")

[==================================================] 100.0% 198.8/198.8MB downloaded


In [ ]:
new_asp_set = []

for cat, word in tqdm(asp_set):
    if word in model.vocab:
        sim = model.most_similar(positive=[f'{word}'])
        for tup in sim:
            if tup[1] > 0.7:
                new_asp_set.append((cat, tup[0]))

  0%|          | 0/1092 [00:00<?, ?it/s]

In [ ]:
asp_set.extend(new_asp_set)

In [ ]:
asp_set = list(set([tuple(t) for t in asp_set ]))

In [ ]:
len(asp_set)

1310

In [ ]:
new_asp_li = []
for cat, word in asp_set:
    new_asp_li.append(word.replace('::', ' ').split('_')[0])

In [ ]:
new_asp_li[:10]

['заказ принять',
 'смузи овощной',
 'филе норвежский лосось с овощ и креветка',
 'ресторан "эскалибур"',
 'тирам',
 'ресторан "оливье"',
 'март',
 'сервировка блюдо',
 'свинин',
 'попробовать']

In [ ]:
import numpy as np
def define_category(aspect):
  classes = ['Whole', 'Food', 'Interior', 'Service', 'Price']
  base_words = ['ресторан_NOUN', 'еда_NOUN', 'интерьер_NOUN', 'обслуживание_NOUN', 'цена_NOUN']
  base_vectors = []
  for c, word in zip(classes, base_words):
      base_vectors.append(model[word])
  if aspect in model.vocab:
      sim = model.cosine_similarities(model[aspect], base_vectors)
      return classes[np.argmax(sim)]
  else:
      return 'Whole'

In [ ]:
import stanza
stanza.download('ru')
nlp = stanza.Pipeline('ru', processors='tokenize')

INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |

INFO:stanza:Use device: gpu
INFO:stanza:Loading: tokenize
INFO:stanza:Done loading processors!


In [ ]:
from collections import defaultdict
aspects2reviews_pred_2 = defaultdict(list)
from nltk.tokenize import word_tokenize
for num, review in test_reviews.items():
    processed = nlp(review)
    for token in processed.iter_tokens():
        lemma = m.parse(token.text.lower())[0].normal_form
        if lemma in new_asp_li:
            if lemma in aspect2cat_fin:
                aspects2reviews_pred_2[num].append({'aspect': lemma, 'cat': aspect2cat_fin[lemma], 'start': token.start_char, 'end': token.end_char})
            else:
                lemma_pos = lemma + '_' + m.parse(token.text.lower())[0].tag.POS
                aspects2reviews_pred_2[num].append({'aspect': lemma, 'cat': define_category(lemma_pos), 'start': token.start_char, 'end': token.end_char})

In [ ]:
len(aspects2reviews_pred_2)

71

In [ ]:
len(aspects2reviews_test)

71

In [ ]:
aspects2reviews_pred_2

defaultdict(list,
            {'13823': [{'aspect': 'сентябрь',
               'cat': 'Service',
               'start': 74,
               'end': 82},
              {'aspect': 'встретить',
               'cat': 'Service',
               'start': 138,
               'end': 146},
              {'aspect': 'менеджер',
               'cat': 'Service',
               'start': 147,
               'end': 155},
              {'aspect': 'девушка',
               'cat': 'Service',
               'start': 179,
               'end': 186},
              {'aspect': 'проводить',
               'cat': 'Service',
               'start': 188,
               'end': 197},
              {'aspect': 'стол', 'cat': 'Interior', 'start': 200, 'end': 205},
              {'aspect': 'меню', 'cat': 'Food', 'start': 213, 'end': 217},
              {'aspect': 'официантка',
               'cat': 'Service',
               'start': 242,
               'end': 252},
              {'aspect': 'заказ', 'cat': 'Service', 'sta

In [ ]:
with open('all_aspects.txt', 'w') as f:
    for k in new_asp_li:
        f.write(k)
        f.write('\n')

In [ ]:
asp_set_proof = []
for c, w in asp_set:
    asp_set_proof.append(w.replace('::', ' ').split('_')[0])

In [ ]:
li_added = list(set(new_asp_li) - set(asp_set_proof))

In [ ]:
with open('added.txt', 'w') as f1:
    for el in li_added:
        f1.write(el)
        f1.write('\n')

## Collocations selection (for n-grams from texts)

## Tonality/mood detection for all words

In [ ]:
! wget 'http://www.labinform.ru/pub/rusentilex/rusentilex_2017.txt'

--2022-12-27 20:08:23--  http://www.labinform.ru/pub/rusentilex/rusentilex_2017.txt
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1329241 (1.3M) [text/plain]
Saving to: ‘rusentilex_2017.txt’

rusentilex_2017.txt 100%[===================>]   1.27M   775KB/s    in 1.7s    

2022-12-27 20:08:27 (775 KB/s) - ‘rusentilex_2017.txt’ saved [1329241/1329241]



In [ ]:
tonal_lexic = []

with open('rusentilex_2017.txt', 'r', encoding='utf-8') as f:
    for lines in f:
        tonal_lexic.append(lines)

In [ ]:
tonal_lexic = tonal_lexic[18:]
print(tonal_lexic)

['аборт, Noun, аборт, negative, fact\n', 'абортивный, Adj, абортивный, negative, fact\n', 'абракадабра, Noun, абракадабра, negative, opinion\n', 'абсурд, Noun, абсурд, negative, opinion\n', 'абсурдность, Noun, абсурдность, negative, opinion\n', 'абсурдный, Adj, абсурдный, negative, opinion\n', 'авантюра, Noun, авантюра, negative, opinion\n', 'авантюризм, Noun, авантюризм, negative, opinion\n', 'авантюрист, Noun, авантюрист, negative, opinion\n', 'авантюристический, Adj, авантюристический, negative, opinion\n', 'авантюристка, Noun, авантюристка, negative, opinion\n', 'авантюристский, Adj, авантюристский, negative, opinion\n', 'авантюрный, Adj, авантюрный, negative, opinion\n', 'аварийная защита, NG, аварийный защита, neutral, fact\n', 'аварийность, Noun, аварийность, negative, fact \n', 'аварийный, Adj, аварийный, negative, fact \n', 'авария, Noun, авария, negative, fact\n', 'авиакатастрофа, Noun, авиакатастрофа, negative, fact \n', 'авиакрушение, Noun, авиакрушение, negative, fact\n', 

In [ ]:
tonal_dict = {}
for item in tonal_lexic:
    i = item.split(',')
    tonal_dict[i[0]] = i[3]

print(tonal_dict)

{'аборт': ' negative', 'абортивный': ' negative', 'абракадабра': ' negative', 'абсурд': ' negative', 'абсурдность': ' negative', 'абсурдный': ' negative', 'авантюра': ' negative', 'авантюризм': ' negative', 'авантюрист': ' negative', 'авантюристический': ' negative', 'авантюристка': ' negative', 'авантюристский': ' negative', 'авантюрный': ' negative', 'аварийная защита': ' neutral', 'аварийность': ' negative', 'аварийный': ' negative', 'авария': ' negative', 'авиакатастрофа': ' negative', 'авиакрушение': ' negative', 'авианалет': ' negative', 'авиаудар': ' negative', 'авиационный налет': ' negative', 'авитаминоз': ' negative', 'авитаминозный': ' negative', 'аврал': ' negative', 'авральный': ' negative', 'автовор': ' negative', 'автоматная очередь': ' negative', 'автоматный огонь': ' negative', 'автомобильная пробка': ' negative', 'автоподстава': ' negative', 'авторитаризм': ' negative', 'авторитарность': ' negative', 'авторитарный': ' negative', 'авторитет падает': ' negative', 'автор

In [ ]:
with open('all_aspects.txt', 'r', encoding='utf-8') as f:
    for lines in f:
        print(lines)

прощание

фонарь

выбор по меню и напиток

директор

скрипач

салат с куриный печень

столик заказать

стена

ресторан "долина"

сервировка блюдо

аппетит

ролл без рис

лепёшка

свинина в кисло-сладкий соус

сентябрь

переспрашивать

грузинский танцевальный коллектив

местечко в угол

планета сушить

азиатский такой кухня

утка

сельдь под шубой…с краб

качество напиток

паста с морепродукт

картофель айдахо

холодное закуска

рибай

шампанское

кольцо кальмар в кляр

огурец

потанцевать

бутылка

густера

холодное напиток

голос солистка

по цена

салат "кавказский"

салат из запечёный овощ

буженина на гриль

сыр филадельфия

суп чанахи

котлетка из телятина с пюре

заказ столик

банкет

вечером

дамский комната

выбор блюдо

аквариум

ризотто с спраж и горгонзола

поступить

кресло

вино

2ом этаж

панакот

саксофонист

молодой девушка

основный блюдо

выполнение заказ

малиновый "аптекарское"

mon petit cafe

администратор екатерина

выбор

черника

закуска к пиво

интерьер зал

в